# Подготовка

## Установка библиотек

In [ ]:
%%capture
!pip install ultralytics rasterio geopandas shapely scikit-learn numpy matplotlib tensorflow osmnx gdown albumentations
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import geopandas as gpd
import osmnx as ox
from rasterio.features import rasterize
from pyproj import Transformer
from rasterio.crs import CRS
from pathlib import Path
from rasterio.windows import Window
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
import shutil
from google.colab import drive
import gdown
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from collections import defaultdict
from pathlib import Path
import torch
import torch.nn as nn
from ultralytics import YOLO
from shapely.geometry import shape
from shapely.geometry import box
import albumentations as A
from albumentations.pytorch import ToTensorV2

## Создание структуры проекта

In [ ]:
os.makedirs('downloaded', exist_ok=True)
os.makedirs('tiles', exist_ok=True)
os.makedirs('buildings', exist_ok=True)
os.makedirs('roads', exist_ok=True)
os.makedirs('pitches', exist_ok=True)
os.makedirs('playgrounds', exist_ok=True)

# Подготовка спутниковых снимков

## Скачивание датасета с Google Drive

In [ ]:
folder_id = "1MJCK7KOr8nBZNVfCRd0M8UQFjdrVj2Ad"
os.makedirs("downloaded", exist_ok=True)

gdown.download_folder(
    id=folder_id,
    output="downloaded",
    quiet=False,
    use_cookies=False
)

## Разделение снимков на плитки

In [ ]:
num_tiles_x = 9  # плитки по горизонтали
num_tiles_y = 3  # плитки по вертикали

folder_path = Path('/kaggle/working/downloaded')

# Перебор всех файлов TIFF
for file_path in folder_path.glob('*.tif*'):
    path_str = str(file_path)
    with rasterio.open(file_path) as src:
        height, width = src.height, src.width
        profile = src.profile

        # Расчёт размеров плиток
        tile_height = height // num_tiles_y
        tile_width = width // num_tiles_x
        remainder_height = height % num_tiles_y
        remainder_width = width % num_tiles_x

        for i in range(num_tiles_y):
            for j in range(num_tiles_x):
                # Расчёт текущего размера плитки (для последних плиток добавляем остаток)
                current_height = tile_height + (remainder_height if i == num_tiles_y - 1 else 0)
                current_width = tile_width + (remainder_width if j == num_tiles_x - 1 else 0)

                # Определение окна для текущей плитки
                window = Window(j * tile_width, i * tile_height, current_width, current_height)

                # Чтение данных для текущей плитки
                tile_data = src.read([1, 2, 3], window=window, boundless=True)

                # Получение трансформации для текущей плитки
                tile_transform = src.window_transform(window)

                # Обновление профиля для текущей плитки
                tile_profile = profile.copy()
                tile_profile.update({
                    'height': current_height,
                    'width': current_width,
                    'transform': tile_transform,
                    'count': 3  # Указываем, что изображение имеет 3 канала
                })

                tile_filename = str(file_path).replace('downloaded/', 'tiles/tile_' + str(i) + '_' + str(j) + '_')

                # Запись плитки в новый файл
                with rasterio.open(tile_filename, 'w', **tile_profile) as dst:
                    dst.write(tile_data)

                print(f"Создан файл: {tile_filename}")

    print(f"Разделение завершено. Создано {num_tiles_x * num_tiles_y} плиток.")

## Отображение всех плиток

In [ ]:
tiles_folder = Path('/kaggle/working/tiles')

if not tiles_folder.exists():
    print(f"Ошибка: Папка {tiles_folder} не найдена.")
else:
    # Поиск всех TIFF-файлов
    tiff_files = list(tiles_folder.glob('*.tif*'))

    # Проверка, есть ли файлы
    if not tiff_files:
        print(f"В папке {tiles_folder} нет TIFF-файлов.")
    else:
        # Группировка тайлов по исходным снимкам
        tile_groups = defaultdict(list)
        for file_path in tiff_files:
            # Имя файла: tile_i_j_suffix.tif
            parts = file_path.stem.split('_')
            if len(parts) >= 3 and parts[0] == 'tile' and parts[1].isdigit() and parts[2].isdigit():
                # Суффикс — всё после tile_i_j
                suffix = '_'.join(parts[3:])
                tile_groups[suffix].append(file_path)
            else:
                print(f"Пропущен файл с некорректным именем: {file_path.name}")

        # Проверка групп
        if not tile_groups:
            print("Не найдено тайлов с корректными именами (tile_i_j_suffix.tif).")
        else:
            print(f"Найдено {len(tile_groups)} исходных изображений.")

            # Отображение каждого исходного изображения
            for suffix, files in sorted(tile_groups.items()):
                # Сортировка тайлов по индексам i, j
                files = sorted(files, key=lambda x: (int(x.stem.split('_')[1]), int(x.stem.split('_')[2])))

                # Проверка количества тайлов
                if len(files) != 27:
                    print(f"Предупреждение: Для снимка '{suffix}' найдено {len(files)} тайлов вместо 27. Пропуск.")
                    continue

                # Настройка фигуры: 3 строки, 9 столбцов
                fig, axes = plt.subplots(3, 9, figsize=(18, 6))  # Размер для читаемости
                axes = axes.flatten()

                # Загрузка и отображение тайлов
                for idx, file_path in enumerate(files):
                    try:
                        with rasterio.open(file_path) as src:
                            # Чтение изображения (RGB или одноканальное)
                            num_bands = src.count
                            if num_bands >= 3:
                                img = src.read([1, 2, 3])  # RGB
                            else:
                                img = src.read(1)
                                img = np.stack([img]*3, axis=0)  # Повторить канал
                            img = np.transpose(img, (1, 2, 0))  # Перевод в HWC

                        # Отображение тайла
                        axes[idx].imshow(img)
                        axes[idx].set_title(file_path.name, fontsize=8)
                        axes[idx].axis('off')
                    except rasterio.errors.RasterioIOError as e:
                        print(f"Ошибка при открытии {file_path.name}: {e}")
                        axes[idx].set_title(f"{file_path.name}\n(Ошибка)", fontsize=8)
                        axes[idx].axis('off')

                # Настройка компоновки
                plt.tight_layout()
                plt.suptitle(f"Исходное изображение: {suffix or 'default'}", fontsize=12, y=1.02)
                plt.show()

            print(f"Отображено {len([g for g in tile_groups.values() if len(g) == 27])} исходных изображений из папки {tiles_folder}.")

## Фильтрация плиток

Некоторые плитки содержат много лишнего: промзоны, частный сектор, торговые центры с парковками, высокие новостройки, Неву.
Мне нужны только те плитки, на которых преобладает застройка зданиями сталинского, хрущевского и брежневского периодов +- 5 этажей

In [ ]:
tiles_folder = Path('/kaggle/working/tiles')

# Словарь для удаления тайлов
# Ключи — названия исходных файлов, значения — списки номеров тайлов (например, '1_0')
# Словарь заполняется вручную
tiles_to_delete = {
    'avtovo_5_2020.tif': ['0_0', '0_1', '0_2', '1_0', '1_1', '1_2', '2_0', '2_1',\
                          '2_2', '0_3', '0_4', '1_6', '0_7', '0_8', '1_7', '1_8', '2_7', '2_8'],
    'izmaylovo_9_2023.tif': ['0_5', '2_0'],
    'kras_pravy_bereg_5_2021.tif': ['0_0', '0_1', '0_2', '0_3', '0_4', '0_7', '0_8',\
                                    '1_0', '1_1', '1_6', '1_7', '1_8', '2_0', '2_1', '2_2', '2_7', '2_8'],
    'leninskiy_prospect_9_2024.tif': ['0_0', '0_1', '0_2', '0_6', '0_7', '0_8', '1_0', '1_8'],
    'moskovskiy_district.tif': ['0_0', '2_0', '2_1', '1_6', '2_6', '2_7', '0_8'],
    'novocherkasskaya_10_2020.tif': ['0_0', '1_0', '2_0', '2_1', '0_1', '2_6', '2_7', '2_8', '1_7', '1_8', '0_7', '0_8'],
    'nsk_levy_bereg_4_2022.tif': ['2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_8', '1_0', '1_1', '1_2', '0_0', '0_1'],
    'akademichesky_9_2024.tif': ['0_8', '2_4', '2_5', '1_4', '1_5'],
    'black_river_4_2019': ['0_0', '0_1', '0_2', '0_3', '1_1', '1_7', '1_8', '2_3', '2_4', '2_5', '2_7', '2_8'],
    'gagarinsky_10_2021': ['0_0', '0_1', '0_2', '0_3', '0_4', '1_0', '2_3', '2_4', '2_5'],
    'kras_pravy_bereg_3_2022': ['0_0', '0_1', '0_2', '0_3', '1_0', '1_1', '1_2', '1_8', '2_4', '2_5', '2_7', '2_8'],
    'lomonosovsky_10_2021': ['0_0', '0_1', '0_7', '1_0', '1_6', '1_8', '2_7', '2_8'],
    'moskovsky_district_5_2020.tif': ['1_3', '1_7', '2_0', '2_6', '2_7'],
    'novocherkasskaya_4_2019.tif': ['0_0', '0_1', '1_0', '2_0', '2_1', '2_2', '2_3', '2_4', '2_7', '2_8', '1_8', '0_8'],
    'petrovsky_park.tif': ['0_2', '1_3', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_8']
}

if not tiles_folder.exists():
    print(f"Ошибка: Папка {tiles_folder} не найдена.")
else:
    deleted_count = 0
    total_files = 0

    # Перебор словаря
    for source_file, tile_indices in tiles_to_delete.items():
        if not tile_indices:
            print(f"Для снимка '{source_file}' не указаны тайлы для удаления.")
            continue

        # Суффикс из имени файла (без .tif)
        suffix = Path(source_file).stem  # Например, 'avtovo_5_2020'

        # Перебор тайлов для удаления
        for tile_index in tile_indices:
            # Формирование имени файла: tile_i_j_suffix.tif
            tile_filename = f"tile_{tile_index}_{suffix}.tif"
            file_path = tiles_folder / tile_filename
            total_files += 1

            try:
                if file_path.exists():
                    file_path.unlink()
                    print(f"Удалён файл: {tile_filename}")
                    deleted_count += 1
                else:
                    print(f"Файл не найден: {tile_filename}")
            except Exception as e:
                print(f"Ошибка при удалении {tile_filename}: {e}")

    if total_files == 0:
        print("Не указаны тайлы для удаления.")
    else:
        print(f"Удаление завершено. Удалено {deleted_count} из {total_files} файлов.")

## Повторное отображение плиток

In [ ]:
tiles_folder = Path('/kaggle/working/tiles')

if not tiles_folder.exists():
    print(f"Ошибка: Папка {tiles_folder} не найдена.")
else:
    # Поиск всех TIFF-файлов
    tiff_files = list(tiles_folder.glob('*.tif*'))

    if not tiff_files:
        print(f"В папке {tiles_folder} нет TIFF-файлов.")
    else:
        # Группировка тайлов по исходным снимкам
        tile_groups = defaultdict(list)
        for file_path in tiff_files:
            # Имя файла: tile_i_j_suffix.tif
            parts = file_path.stem.split('_')
            if len(parts) >= 3 and parts[0] == 'tile' and parts[1].isdigit() and parts[2].isdigit():
                suffix = '_'.join(parts[3:])  # Суффикс — всё после tile_i_j
                tile_groups[suffix].append(file_path)
            else:
                print(f"Пропущен файл с некорректным именем: {file_path.name}")

        # Проверка групп
        if not tile_groups:
            print("Не найдено тайлов с корректными именами (tile_i_j_suffix.tif).")
        else:
            print(f"Найдено {len(tile_groups)} исходных изображений.")
            displayed_images = 0

            # Отображение каждого исходного изображения
            for suffix, files in sorted(tile_groups.items()):
                # Сортировка тайлов по индексам i, j
                files = sorted(files, key=lambda x: (int(x.stem.split('_')[1]), int(x.stem.split('_')[2])))

                # Создание карты тайлов для сетки 3x9
                tile_map = {(i, j): None for i in range(3) for j in range(9)}
                for file_path in files:
                    parts = file_path.stem.split('_')
                    i, j = int(parts[1]), int(parts[2])
                    if (i, j) in tile_map:
                        tile_map[(i, j)] = file_path
                    else:
                        print(f"Некорректные индексы для {file_path.name}")

                # Настройка фигуры: 3 строки, 9 столбцов
                fig, axes = plt.subplots(3, 9, figsize=(18, 6))
                axes = axes.flatten()

                # Заполнение сетки
                idx = 0
                for i in range(3):
                    for j in range(9):
                        file_path = tile_map.get((i, j))
                        if file_path:
                            try:
                                with rasterio.open(file_path) as src:
                                    # Чтение изображения (RGB или одноканальное)
                                    num_bands = src.count
                                    if num_bands >= 3:
                                        img = src.read([1, 2, 3])  # RGB
                                    else:
                                        img = src.read(1)
                                        img = np.stack([img]*3, axis=0)  # Повторить канал
                                    img = np.transpose(img, (1, 2, 0))  # Перевод в HWC

                                # Отображение тайла
                                axes[idx].imshow(img)
                                axes[idx].set_title(file_path.name, fontsize=8)
                                axes[idx].axis('off')
                            except rasterio.errors.RasterioIOError as e:
                                print(f"Ошибка при открытии {file_path.name}: {e}")
                                axes[idx].set_title(f"{file_path.name}\n(Ошибка)", fontsize=8)
                                axes[idx].axis('off')
                        else:
                            # Пустая ячейка для отсутствующего тайла
                            axes[idx].set_title(f"tile_{i}_{j}_{suffix}\n(Отсутствует)", fontsize=8)
                            axes[idx].axis('off')
                        idx += 1

                # Настройка компоновки
                plt.tight_layout()
                plt.suptitle(f"Исходное изображение: {suffix or 'default'}", fontsize=12, y=1.02)
                plt.show()
                displayed_images += 1

            print(f"Отображено {displayed_images} исходных изображений из папки {tiles_folder}.")

# Разметка зданий

## Создание масок зданий

In [ ]:
def create_building_mask(image_path, output_mask_path):
    
    if os.path.exists(output_mask_path):
        print(f"Маска зданий уже существует: {output_mask_path}, пропускаем.")
        return
        
    # Загрузка спутникового снимка для получения формы и трансформации
    with rasterio.open(image_path) as src:
        image = src.read()
        transform = src.transform
        crs = src.crs
        height, width = src.height, src.width
        bounds = src.bounds

        # Преобразование координат в EPSG:4326 (широта, долгота)
        transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
        west, south = transformer.transform(bounds.left, bounds.bottom)
        east, north = transformer.transform(bounds.right, bounds.top)

    # Загрузка данных о зданиях с помощью osmnx
    tags = {'building': True}
    exclude_tags = ['garage', 'garages'] #Гаражи не берем
    
    bounds = [west, south, east, north]
    gdf = ox.features.features_from_bbox(bounds, tags=tags)
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]  # Фильтрация только полигонов
    gdf = gdf[~gdf['building'].isin(exclude_tags)]
    gdf = gdf.to_crs(crs)

    # Растеризация зданий
    shapes = ((geom, 1) for geom in gdf.geometry)
    mask = rasterize(shapes, out_shape=(height, width), transform=transform, fill=0, dtype='uint8')

    # Сохранение маски в формате GeoTIFF
    with rasterio.open(output_mask_path, 'w', driver='GTiff', height=height, width=width, count=1, dtype='uint8', crs=crs, transform=transform) as dst:
        dst.write(mask, 1)

In [ ]:
%%capture

folder_path = Path('/kaggle/working/tiles')

# Перебор всех файлов TIFF
for file_path in folder_path.glob('*.tif*'):
    path_str = str(file_path)
    create_building_mask(file_path, path_str.replace('.tif', '_buildings.tif').replace('/tiles', '/buildings'))

## Класс датасета для зданий

In [ ]:
class BuildingDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.tif')]  # Предполагается, что файлы имеют расширение .tif
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.tif', '_buildings.tif'))
        image = Image.open(img_path).convert("RGB")  # Загрузка изображения в формате RGB
        mask = Image.open(mask_path).convert("L")    # Загрузка маски в градациях серого
        
        # Преобразование PIL Images в массивы NumPy
        image_np = np.array(image)
        mask_np = np.array(mask)
        
        if self.transform:
            augmented = self.transform(image=image_np, mask=mask_np)
            image = augmented['image']
            mask = augmented['mask']

        image = image.float()
        mask = mask.float()
        
        return image, mask

# Разметка дорог

## Создание масок дорог

In [ ]:
def create_road_mask(tile_path, output_path, buffer=5, bbox_buffer=0.01):

    if os.path.exists(output_path):
        print(f"Маска дорог уже существует: {output_path}, пропускаем.")
        return
    
    with rasterio.open(tile_path) as src:
        bounds = src.bounds
        src_crs = src.crs
        transform = src.transform
        width, height = src.width, src.height

    # Преобразование границ в географические координаты (EPSG:4326)
    transformer_to_4326 = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
    minx, miny = transformer_to_4326.transform(bounds.left, bounds.bottom)
    maxx, maxy = transformer_to_4326.transform(bounds.right, bounds.top)

    # Добавление буфера к границам (~1000 метров в градусах)
    minx, miny = minx - bbox_buffer, miny - bbox_buffer
    maxx, maxy = maxx + bbox_buffer, maxy + bbox_buffer

    # Запрос дорог из OSM
    try:
        G = ox.graph_from_bbox((minx, miny, maxx, maxy), network_type='drive')
        edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

        # Определение подходящей проекционной CRS (UTM)
        utm_crs = edges.estimate_utm_crs()

        # Преобразование геометрий в UTM для буферизации
        edges = edges.to_crs(utm_crs)
        edges['geometry'] = edges['geometry'].buffer(buffer)

        # Преобразование обратно в исходную CRS тайла
        edges = edges.to_crs(src_crs)

        # Растеризация геометрий дорог
        shapes = ((geom, 1) for geom in edges['geometry'])
        mask = rasterize(shapes, out_shape=(height, width), transform=transform, fill=0, dtype=np.uint8)

    except ValueError as e:
        if "Graph contains no edges" in str(e) or "Found no graph nodes within the requested polygon" in str(e):
            # Создать пустую маску, если дорог или узлов нет
            mask = np.zeros((height, width), dtype=np.uint8)
        else:
            raise e

    # Сохранение маски
    with rasterio.open(
        output_path, 'w', driver='GTiff', height=height, width=width, count=1,
        dtype='uint8', crs=src_crs, transform=transform
    ) as dst:
        dst.write(mask, 1)

In [ ]:
%%capture

folder_path = Path('/kaggle/working/tiles')

# Перебор всех файлов TIFF
for file_path in folder_path.glob('*.tif*'):
    path_str = str(file_path)
    create_road_mask(file_path, path_str.replace('.tif', '_road_mask.tif').replace('/tiles', '/roads'))

## Класс датасета для дорог

In [ ]:
class RoadDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.tif')]  # Предполагается, что файлы имеют расширение .tif
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.tif', '_road_mask.tif'))
        image = Image.open(img_path).convert("RGB")  # Загрузка изображения в формате RGB
        mask = Image.open(mask_path).convert("L")    # Загрузка маски в градациях серого
        
        # Преобразование PIL Images в массивы NumPy
        image_np = np.array(image)
        mask_np = np.array(mask)
        
        # Применение трансформации, если она задана
        if self.transform:
            augmented = self.transform(image=image_np, mask=mask_np)
            image = augmented['image']
            mask = augmented['mask']

        image = image.float()
        mask = mask.float()
        
        return image, mask

## Определение классов дворовых объектов

In [ ]:
classes = ['playground', 'sports_field']
num_classes = len(classes)

# Предобработка данных

## Трансформации

In [ ]:
mean=[0, 0, 0]
std=[1, 1, 1]
max_pixel_value=255.0

train_transform = A.Compose([
    A.RandomRotate90(p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5, brightness_limit=0.2, contrast_limit=0.2),
    A.Resize(1024, 1024),
    #A.Normalize(mean=mean, std=std, max_pixel_value=max_pixel_value),
    ToTensorV2(),
], additional_targets={'mask': 'mask'})

val_transform = A.Compose([
    A.Resize(1024, 1024),
    #A.Normalize(mean=mean, std=std, max_pixel_value=max_pixel_value),
    ToTensorV2(),
], additional_targets={'mask': 'mask'})

## Разделение на train и val

In [ ]:
images_dir = 'tiles'
playgrounds_dir = 'playgrounds'
sports_dir = 'pitches'
building_mask_dir = 'buildings'
road_mask_dir = 'roads'

images = [os.path.join(images_dir, img) for img in os.listdir(images_dir) if img.endswith('.tif')]
road_masks = [os.path.join(road_mask_dir, img.replace('.tif', '_road_mask.tif')) for img in os.listdir(images_dir) if img.endswith('.tif')]
building_masks = [os.path.join(building_mask_dir, img.replace('.tif', '_buildings.tif')) for img in os.listdir(images_dir) if img.endswith('.tif')]

# Разделение на тренировочный и валидационный наборы (90% - 10%)
train_images, val_images, train_road_masks, val_road_masks, train_building_masks, val_building_masks = train_test_split(
    images, road_masks, building_masks, test_size=0.1, random_state=42)

os.makedirs('images/train', exist_ok=True)
os.makedirs('images/val', exist_ok=True)
os.makedirs('labels/train', exist_ok=True)
os.makedirs('labels/val', exist_ok=True)

# Копирование изображений
for file in train_images:
    shutil.copy(str(file), str(file).replace('tiles', 'images/train'))
for file in val_images:
    shutil.copy(str(file), str(file).replace('tiles', 'images/val'))

## Создание датасетов и загрузчиков

In [ ]:
train_images_dir = '/kaggle/working/images/train'
val_images_dir = '/kaggle/working/images/val'
roads_dir = '/kaggle/working/roads'
buildings_dir = '/kaggle/working/buildings'

# Создание датасетов
train_building_dataset = BuildingDataset(train_images_dir, buildings_dir, transform=train_transform)
val_building_dataset = BuildingDataset(val_images_dir, buildings_dir, transform=val_transform)
train_road_dataset = RoadDataset(train_images_dir, roads_dir, transform=train_transform)
val_road_dataset = RoadDataset(val_images_dir, roads_dir, transform=val_transform)

# Создание загрузчиков данных
train_roads_loader = DataLoader(train_road_dataset, batch_size=1, shuffle=True)
val_roads_loader = DataLoader(val_road_dataset, batch_size=1, shuffle=False)
train_buildings_loader = DataLoader(train_building_dataset, batch_size=1, shuffle=True)
val_buildings_loader = DataLoader(val_building_dataset, batch_size=1, shuffle=False)

# UNet и функции потерь

## Класс UNet

In [ ]:
# Очистка памяти GPU перед загрузкой модели
torch.cuda.empty_cache()

# Двойной свёрточный блок
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

# Слой понижения разрешения
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.mpconv(x)

# Слой повышения разрешения
class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([skip, x], dim=1)
        return self.conv(x)

# Выходной свёрточный слой
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

# U-Net для изображений 1024x1024
class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        # Энкодер с уменьшенными каналами
        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 512)
        # Бутылочное горлышко
        self.bottleneck = DoubleConv(512, 512)
        # Декодер
        self.up1 = Up(512, 256)
        self.up2 = Up(256, 128)
        self.up3 = Up(128, 64)
        self.up4 = Up(64, 32)
        # Выходной слой
        self.outc = OutConv(32, n_classes)

    def forward(self, x):
        # Энкодер
        x1 = self.inc(x)        # 32, 1024, 1024
        x2 = self.down1(x1)     # 64, 512, 512
        x3 = self.down2(x2)     # 128, 256, 256
        x4 = self.down3(x3)     # 256, 128, 128
        x5 = self.down4(x4)     # 512, 64, 64
        # Бутылочное горлышко
        x = self.bottleneck(x5) # 512, 64, 64
        # Декодер
        x = self.up1(x, x4)     # 256, 128, 128
        x = self.up2(x, x3)     # 128, 256, 256
        x = self.up3(x, x2)     # 64, 512, 512
        x = self.up4(x, x1)     # 32, 1024, 1024
        # Выход
        logits = self.outc(x)   # n_classes, 1024, 1024
        return logits

## Dice Loss

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs).view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

In [ ]:
def dice_coefficient(pred, target, smooth=1):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

## Focal Loss

In [ ]:
def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.75,
    gamma: float = 2.0,
    reduction: str = "mean",
) -> torch.Tensor:
    p = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

# Обучение UNet для зданий

## Инициализация модели

In [ ]:
model = UNet(n_channels=3, n_classes=1)
criterion = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=3, verbose=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
device

## Цикл обучения

In [ ]:
# Инициализация для смешанной точности
scaler = GradScaler()

# Параметры накопления градиентов
accumulation_steps = 12

# Параметры ранней остановки
best_val_dice = 0
patience = 20
counter = 0

# Цикл обучения
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    torch.cuda.empty_cache()
    running_loss = 0.0
    optimizer.zero_grad()
    
    # Тренировочный цикл
    for i, (images, masks) in enumerate(train_buildings_loader):
        images, masks = images.to(device), masks.to(device)
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, masks) / accumulation_steps
        scaler.scale(loss).backward()
        
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        running_loss += loss.item() * accumulation_steps
    
    # Обработка оставшихся накопленных градиентов
    if (i + 1) % accumulation_steps != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
    
    # Валидационный цикл
    model.eval()
    val_loss = 0.0
    val_dice_scores = []
    with torch.no_grad():
        for images, masks in val_buildings_loader:
            images, masks = images.to(device), masks.to(device)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, masks)
            val_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            for pred, mask in zip(preds, masks):
                dice = dice_coefficient(pred, mask)
                val_dice_scores.append(dice.item())
    epoch_val_loss = val_loss / len(val_buildings_loader)
    epoch_val_dice = np.mean(val_dice_scores)
    
    # Вывод потерь и метрик
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_buildings_loader):.4f}, Val Loss: {epoch_val_loss:.4f}, Val Dice: {epoch_val_dice:.4f}')
    
    # Шаг планировщика скорости обучения
    scheduler.step(epoch_val_dice)
    
    # Ранняя остановка
    if epoch_val_dice > best_val_dice:
        best_val_dice = epoch_val_dice
        counter = 0
        torch.save(model.state_dict(), 'best_buildings_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f'Ранняя остановка на эпохе {epoch+1}')
            break

## Тестирование и визуализация

In [ ]:
def visualize(image, mask, prediction):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image.permute(1, 2, 0).numpy())
    ax[0].set_title('Изображение')
    ax[1].imshow(mask.squeeze().numpy(), cmap='gray')
    ax[1].set_title('Истинная маска')
    ax[2].imshow(prediction.squeeze().numpy(), cmap='gray')
    ax[2].set_title('Предсказанная маска')
    plt.show()

torch.cuda.empty_cache()  # Очистка памяти перед тестированием
model.eval()
with torch.no_grad():
    for images, masks in val_buildings_loader:
        images = images.to(device)
        masks = masks.to(device)
        with autocast():
            outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()

        # Визуализация одного примера
        image = (images[0] / 255).cpu()
        mask = masks[0].cpu()
        pred = preds[0].cpu()
        visualize(image, mask, pred)
        #break  # Показать только один пример

def dice_coefficient(pred, target):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + 1) / (pred.sum() + target.sum() + 1)

model.eval()
dice_scores = []
with torch.no_grad():
    for images, masks in val_buildings_loader:
        images = images.to(device)
        masks = masks.to(device)
        with autocast():
            outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        for pred, mask in zip(preds, masks):
            dice = dice_coefficient(pred, mask)
            dice_scores.append(dice.item())

print(f'Средний Dice Coefficient на валидационной выборке: {np.mean(dice_scores)}')

# Обучение UNet для дорог

## Инициализация модели

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Инициализация модели
model = UNet(n_channels=3, n_classes=1).to(device)
criterion = lambda inputs, targets: sigmoid_focal_loss(inputs, targets, alpha=0.75, gamma=2.0)
optimizer = optim.Adam(model.parameters(), lr=2e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=5, verbose=True)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
device

## Цикл обучения

In [ ]:
# Инициализация для смешанной точности
scaler = GradScaler()

# Параметры накопления градиентов
accumulation_steps = 12

# Параметры ранней остановки
best_val_dice = 0
patience = 20
counter = 0

# Цикл обучения
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    torch.cuda.empty_cache()
    running_loss = 0.0
    optimizer.zero_grad()
    
    # Тренировочный цикл
    for i, (images, masks) in enumerate(train_roads_loader):
        images, masks = images.to(device), masks.to(device)
        masks = masks.unsqueeze(1)
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, masks) / accumulation_steps
        scaler.scale(loss).backward()
        
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        running_loss += loss.item() * accumulation_steps
    
    # Обработка оставшихся накопленных градиентов
    if (i + 1) % accumulation_steps != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
    
    # Валидационный цикл
    model.eval()
    val_loss = 0.0
    val_dice_scores = []
    with torch.no_grad():
        for images, masks in val_roads_loader:
            images, masks = images.to(device), masks.to(device)
            masks = masks.unsqueeze(1)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, masks)
            val_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            for pred, mask in zip(preds, masks):
                dice = dice_coefficient(pred, mask)
                val_dice_scores.append(dice.item())
    epoch_val_loss = val_loss / len(val_roads_loader)
    epoch_val_dice = np.mean(val_dice_scores)
    
    # Вывод потерь и метрик
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_roads_loader):.4f}, Val Loss: {epoch_val_loss:.4f}, Val Dice: {epoch_val_dice:.4f}')
    
    # Шаг планировщика скорости обучения
    scheduler.step(epoch_val_dice)
    
    # Ранняя остановка
    if epoch_val_dice > best_val_dice:
        best_val_dice = epoch_val_dice
        counter = 0
        torch.save(model.state_dict(), 'best_roads_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f'Ранняя остановка на эпохе {epoch+1}')
            break

## Тестирование и визуализация

In [ ]:
def visualize(image, mask, prediction):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image.permute(1, 2, 0).numpy())
    ax[0].set_title('Изображение')
    ax[1].imshow(mask.squeeze().numpy(), cmap='gray')
    ax[1].set_title('Истинная маска дороги')
    ax[2].imshow(prediction.squeeze().numpy(), cmap='gray')
    ax[2].set_title('Предсказанная маска дороги')
    plt.show()

torch.cuda.empty_cache()
model.eval()
with torch.no_grad():
    for images, masks in val_roads_loader:
        images, masks = images.to(device), masks.to(device)
        with autocast():
            outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        image = images[0].cpu()
        mask = masks[0].cpu()
        pred = preds[0].cpu()
        visualize(image, mask, pred)
        #break

# Расчёт метрик
def dice_coefficient(pred, target):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + 1) / (pred.sum() + target.sum() + 1)

model.eval()
dice_scores = []
with torch.no_grad():
    for images, masks in val_roads_loader:
        images, masks = images.to(device), masks.to(device)
        with autocast():
            outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        for pred, mask in zip(preds, masks):
            dice = dice_coefficient(pred, mask)
            dice_scores.append(dice.item())
print(f'Средний Dice Coefficient для дорог: {np.mean(dice_scores)}')

# Дообучение YOLOv8 для площадок

## Генерация аннотаций

In [ ]:
# Список снимков, на которых видно площадки (заполняется вручную)
selected_snapshots = [
    'novocherkasskaya_4_2019.tif',
    'black_river_4_2019.tif',
    'moskovsky_district_5_2020.tif',
    'kras_pravy_bereg_3_2022.tif',
    'avtovo_5_2020.tif' #jpg
]

In [ ]:
def get_annotation_line(geom, class_idx, width, height, transform, image_path):
    if geom.geom_type != 'Polygon':
        print(f"Пропущен: геометрия не является полигоном, тип: {geom.geom_type}")
        return []
    
    # Проверка CRS изображения
    with rasterio.open(image_path) as src:
        image_crs = src.crs
        bounds = src.bounds
        image_polygon = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
    
    # Логирование исходной площади и границ
    print(f"Исходная площадь полигона: {geom.area}, bounds={geom.bounds}")
    
    # Проверка валидности геометрии
    if not geom.is_valid:
        print(f"Геометрия невалидна, пытаемся исправить: {geom}")
        geom = geom.buffer(0)
        if not geom.is_valid:
            print(f"Геометрия не может быть исправлена: {geom}")
            return []
    
    # Синхронизация CRS
    try:
        gdf = gpd.GeoDataFrame({'geometry': [geom]}, crs='EPSG:4326')
        if image_crs and image_crs != 'EPSG:4326':
            gdf = gdf.to_crs(image_crs)
            geom = gdf.geometry.iloc[0]
            print(f"Геометрия преобразована в CRS изображения: {image_crs}")
    except Exception as e:
        print(f"Ошибка при преобразовании CRS: {e}")
        return []
    
    # Обрезка полигона
    if not geom.intersects(image_polygon):
        print(f"Полигон не пересекает границы изображения: {geom.bounds}")
        return []
    
    clipped_geom = geom.intersection(image_polygon)
    if clipped_geom.is_empty:
        print(f"Полигон после обрезки пуст: {geom.bounds}")
        return []
    
    if clipped_geom.geom_type != 'Polygon':
        print(f"Обрезанная геометрия не является полигоном, тип: {clipped_geom.geom_type}")
        return []
    
    # Логирование площади после обрезки
    print(f"Площадь после обрезки: {clipped_geom.area}, bounds={clipped_geom.bounds}")
    
    # Получить контуры полигона
    exteriors = [list(clipped_geom.exterior.coords)]
    interiors = [list(int.coords) for int in clipped_geom.interiors]
    all_rings = exteriors + interiors
    
    # Преобразовать точки в пиксельные координаты
    all_points = []
    pixel_y_coords = []  # Для вычисления bounding box
    for ring in all_rings:
        ring_points = []
        for x, y in ring:
            px, py = ~transform * (x, y)
            print(f"Координаты: географические=({x}, {y}), пиксельные=({px}, {py})")
            px_norm = min(max(px / width, 0.0), 1.0)
            py_norm = min(max(py / height, 0.0), 1.0)
            ring_points.append((px_norm, py_norm))
            pixel_y_coords.append(py)
        all_points.extend(ring_points)
    
    # Получить ограничивающий прямоугольник
    minx, miny, maxx, maxy = clipped_geom.bounds
    minx_px, miny_px = ~transform * (minx, miny)
    maxx_px, maxy_px = ~transform * (maxx, maxy)
    
    # Исправление bounding box: явно вычисляем min/max по Y
    pixel_y_min = min(pixel_y_coords)
    pixel_y_max = max(pixel_y_coords)
    print(f"Bounding box в пикселях: x_min={minx_px}, x_max={maxx_px}, y_min={pixel_y_min}, y_max={pixel_y_max}")
    
    # Нормализовать координаты bounding box
    x_center = min(max((minx_px + maxx_px) / 2 / width, 0.0), 1.0)
    y_center = min(max((pixel_y_min + pixel_y_max) / 2 / height, 0.0), 1.0)
    w = min(max((maxx_px - minx_px) / width, 0.0), 1.0)
    h = min(max((pixel_y_max - pixel_y_min) / height, 0.0), 1.0)
    
    # Проверка валидности bounding box
    if w == 0 or h == 0:
        print(f"Некорректный bounding box: w={w}, h={h}, bounds={clipped_geom.bounds}")
        return []
    
    # Формирование аннотации
    line = [str(class_idx), str(x_center), str(y_center), str(w), str(h)]
    line.extend([str(coord) for pair in all_points for coord in pair])
    
    print(f"Создана аннотация: class={class_idx}, x_center={x_center}, y_center={y_center}, w={w}, h={h}, points={len(all_points)}")
    return [' '.join(line)]

## Создание полигонов площадок

In [ ]:
%%capture

images_dir = 'tiles'
playgrounds_dir = 'playgrounds'
sports_dir = 'pitches'
train_images_dir = 'images/train'
val_images_dir = 'images/val'
labels_train_dir = 'labels/train'
labels_val_dir = 'labels/val'

# Функция для проверки существования файлов и фильтрации тайлов по снимкам
def get_selected_tiles(snapshot_list, tiles_dir, train_dir, val_dir):
    train_tiles = []
    val_tiles = []
    tiles_path = Path(tiles_dir)
    train_path = Path(train_dir)
    val_path = Path(val_dir)
    
    # Получение всех тайлов из директории tiles
    all_tiles = [f for f in tiles_path.glob('*.tif*') if f.is_file()]
    
    # Фильтрация тайлов по указанным снимкам
    for tile in all_tiles:
        tile_name = tile.name
        # Проверяем, содержит ли имя тайла суффикс одного из снимков
        for snapshot in snapshot_list:
            snapshot_name = Path(snapshot).stem  # Например, 'avtovo_5_2020'
            if tile_name.endswith(f"_{snapshot_name}.tif"):
                # Проверяем, в какой выборке (train/val) находится тайл
                if (train_path / tile_name).exists():
                    train_tiles.append(tile_name)
                elif (val_path / tile_name).exists():
                    val_tiles.append(tile_name)
    
    return train_tiles, val_tiles

# Получение тайлов для тренировочной и валидационной выборок
train_tiles, val_tiles = get_selected_tiles(selected_snapshots, images_dir, train_images_dir, val_images_dir)

# Проверка, что списки не пусты
if not train_tiles and not val_tiles:
    print("Ошибка: Не найдено тайлов, соответствующих указанным снимкам, или списки пусты.")
else:
    # Проверка аннотаций
    missing_annotations = []
    for split, tiles in [('train', train_tiles), ('val', val_tiles)]:
        labels_dir = labels_train_dir if split == 'train' else labels_val_dir
        labels_path = Path(labels_dir)
        for tile in tiles:
            base_name = Path(tile).stem
            label_file = labels_path / f"{base_name}.txt"
            if not label_file.exists():
                missing_annotations.append((split, tile))
    
    if missing_annotations:
        print("Найдены тайлы без аннотаций. Необходимо перегенерировать аннотации для следующих файлов:")
        for split, tile in missing_annotations:
            print(f"{split}: {tile}")
    else:
        print("Все аннотации для выбранных тайлов присутствуют.")

    # Создание или обновление data.yaml
    with open('data.yaml', 'w') as f:
        f.write('train: images/train\n')
        f.write('val: images/val\n')
        f.write('nc: 2\n')
        f.write('names: ["playground", "sports_field"]\n')

    print(f"Подготовка данных для YOLOv8 завершена.")
    print(f"Тренировочная выборка: {len(train_tiles)} тайлов")
    print(f"Валидационная выборка: {len(val_tiles)} тайлов")

## Отображение примера площадок

In [ ]:
import random
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection

# Пути к данным
images_dir = os.path.join('', 'tiles')
playgrounds_dir = os.path.join('', 'playgrounds')
sports_dir = os.path.join('', 'pitches')

# Получение списка TIFF-файлов
tiff_files = [f for f in os.listdir(images_dir) if f.endswith('.tif')]

if not tiff_files:
    print("Нет TIFF-файлов в папке tiles.")
else:
    random_file = random.choice(tiff_files)
    base_name = os.path.splitext(random_file)[0]
    image_path = os.path.join(images_dir, random_file)
    playgrounds_geojson = os.path.join(playgrounds_dir, f"{base_name}_playgrounds.geojson")
    sports_geojson = os.path.join(sports_dir, f"{base_name}_pitches.geojson")

    # Загрузка снимка
    with rasterio.open(image_path) as src:
        image = src.read([1, 2, 3]).transpose(1, 2, 0)  # RGB
        bounds = src.bounds
        extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

    # Инициализация фигуры
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image, extent=extent)

    # Функция для добавления полигонов на график
    def plot_polygons(gdf, color, label):
        patches = []
        for geom in gdf.geometry:
            if geom is None:
                continue
            if geom.geom_type == 'Polygon':
                coords = list(geom.exterior.coords)
                patches.append(MplPolygon(coords, closed=True, edgecolor=color, fill=False, linewidth=2))
            elif geom.geom_type == 'MultiPolygon':
                for poly in geom.geoms:
                    coords = list(poly.exterior.coords)
                    patches.append(MplPolygon(coords, closed=True, edgecolor=color, fill=False, linewidth=2))
        if patches:
            p = PatchCollection(patches, match_original=True)
            ax.add_collection(p)
            # Добавление пустого патча для легенды
            ax.plot([], [], color=color, label=label, linewidth=2)

    # Загрузка и отображение детских площадок
    if os.path.exists(playgrounds_geojson):
        gdf_playgrounds = gpd.read_file(playgrounds_geojson)
        if not gdf_playgrounds.empty:
            plot_polygons(gdf_playgrounds, 'red', 'Детские площадки')
        else:
            print(f"GeoJSON {playgrounds_geojson} пустой.")
    else:
        print(f"GeoJSON {playgrounds_geojson} не найден.")

    # Загрузка и отображение спортивных площадок
    if os.path.exists(sports_geojson):
        gdf_sports = gpd.read_file(sports_geojson)
        if not gdf_sports.empty:
            plot_polygons(gdf_sports, 'blue', 'Спортивные площадки')
        else:
            print(f"GeoJSON {sports_geojson} пустой.")
    else:
        print(f"GeoJSON {sports_geojson} не найден.")

    # Настройка графика
    ax.set_title(f"Снимок: {random_file}")
    if ax.get_legend_handles_labels()[0]:  # Проверка, есть ли элементы для легенды
        ax.legend()
    plt.xlabel('X (CRS)')
    plt.ylabel('Y (CRS)')
    plt.show()

## Дообучение модели YOLOv8

In [ ]:
# Загрузка предобученной модели
model = YOLO('yolov8n-seg.pt')

# Дообучение
model.train(data='data.yaml', epochs=100, imgsz=1024)

# Сохранение модели
model.save('best.pt')

## Тестирование и визуализация

In [ ]:
model_path = '/kaggle/working/best.pt'

train_images_dir = 'images/train'
val_images_dir = 'images/val'

model = YOLO(model_path)

class_names = ['playground', 'sports_field']

# Функция для обработки изображений из заданной папки
def find_top_10_detected_images(images_dir, split_name, max_images=10):
    image_files = [f for f in Path(images_dir).glob('*.tif') if f.is_file()]
    print(f"Проверка {split_name}: {len(image_files)} изображений")
    
    detected_count = 0
    for image_path in image_files:
        if detected_count >= max_images:
            break
        
        print(f"Обработка изображения: {image_path}")
        
        # Выполнение предсказания
        results = model.predict(str(image_path), conf=0.5, imgsz=960, verbose=False)
        
        # Проверка результатов
        for result in results:
            if result.boxes is not None and len(result.boxes) > 0:  # Если есть обнаруженные объекты
                print(f"Объект обнаружен в {image_path}!")
                
                # Извлечение информации о всех обнаруженных объектах
                boxes = result.boxes.xywh.cpu().numpy()  # Координаты bounding box (x_center, y_center, w, h)
                classes = result.boxes.cls.cpu().numpy()  # Классы
                confidences = result.boxes.conf.cpu().numpy()  # Уверенности
                
                # Вывод информации о каждом объекте
                for box, cls, conf in zip(boxes, classes, confidences):
                    class_name = class_names[int(cls)]
                    x_center, y_center, w, h = box
                    print(f"Класс: {class_name}, Уверенность: {conf:.3f}")
                    print(f"Местоположение (bounding box): x_center={x_center:.1f}, y_center={y_center:.1f}, w={w:.1f}, h={h:.1f}")
                
                # Визуализация результата
                img = cv2.imread(str(image_path))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Конвертация в RGB для matplotlib
                
                # Отрисовка всех bounding box
                for box, cls, conf in zip(boxes, classes, confidences):
                    x_center, y_center, w, h = box
                    x1, y1 = int(x_center - w / 2), int(y_center - h / 2)
                    x2, y2 = int(x_center + w / 2), int(y_center + h / 2)
                    color = (255, 0, 0) if class_names[int(cls)] == 'playground' else (0, 255, 0)  # Красный для playground, зеленый для sports_field
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(img, f"{class_names[int(cls)]} ({conf:.2f})", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                
                # Если есть маска сегментации
                if result.masks is not None and len(result.masks) > 0:
                    for mask in result.masks.data.cpu().numpy():
                        mask = mask.squeeze()  # Удаление лишних размерностей
                        mask = (mask > 0).astype(np.uint8) * 255  # Бинарная маска
                        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))  # Масштабирование маски
                        colored_mask = np.zeros_like(img)
                        colored_mask[:, :, 1] = mask  # Зеленая маска
                        img = cv2.addWeighted(img, 0.8, colored_mask, 0.4, 0)  # Наложение маски
                
                # Отображение изображения
                plt.figure(figsize=(10, 10))
                plt.imshow(img)
                plt.title(f"Обнаруженные объекты в {image_path.name} (№{detected_count + 1})")
                plt.axis('off')
                plt.show()
                
                detected_count += 1
                break  # Переходим к следующему изображению после обработки
    
    if detected_count == 0:
        print(f"В {split_name} не найдено изображений с распознанными объектами.")

# Проверка тренировочной и валидационной выборок
find_top_10_detected_images(train_images_dir, "train", max_images=10)
find_top_10_detected_images(val_images_dir, "val", max_images=10 - min(10, len([f for f in Path(train_images_dir).glob('*.tif') if f.is_file() if os.path.exists(str(f)) and find_top_10_detected_images(train_images_dir, "train", max_images=1)])))